In [1]:
from data_functions import *
from ML_continuous import *

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import statsmodels.formula.api as smf

import random

In [2]:
for_pres = read('texas_president.xlsx')
for_pres = for_pres.rename(index=str, columns={"count": "county"})
for_pres = for_pres.rename(index=str, columns={"Urbanization Status": "urban", "Border Status": "border"})

In [3]:
for_pres.border.value_counts()

Non-Border    222
Border         32
Name: border, dtype: int64

In [4]:
for_house = read('texas_counties.xlsx')
for_house = for_house.rename(index=str, columns={"rep_number": "district_num", "candidate_name": "dem_candidate", "dem_vote_share": "two_party_vtshare", "Urbanization Status": "urban", "Border Status": "border"})
for_house = for_house[for_house.county != 'ALL COUNTIES']
print(for_house.shape)

(308, 4)


In [5]:
counties = list(for_pres.county)

county_nums = {}
val = 1
for c in counties:
    county_nums[c] = val
    val += 1
    
for_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])
for_house['office'] = 'house'
for_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]

## Info data frames will contain all the demographic etc information for each of those entities: county, pres candidate, or house candidate. To be combined after research is finished

In [6]:
county_info = for_pres[['county', 'county_num', 'urban', 'border']]
county_info['urban'] = county_info.urban.apply(lambda x: 1 if x == 'Urban' else 0)
county_info['border'] = county_info.urban.apply(lambda x: 1 if x == 'Border' else 0)

In [7]:
pres_info = for_pres[['office', 'dem_candidate', 'two_party_vtshare', 'candidate_male', 'candidate_white', 'candidate_hispanic']]
#pres_info = pres_candidates.groupby(['dem_candidate','office']).median()
#pres_info = pres_candidates.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [8]:
house_info = for_house[['district_num', 'office', 'dem_candidate', 'two_party_vtshare']]
house_info = house_info.groupby(['dem_candidate','office']).median()
house_info = house_info.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [9]:
house_info.to_csv('house_candidates_to_add_info.csv')

In [10]:
for_house = for_house.merge(county_info, on=['county', 'county_num'])
for_house.columns

Index(['county', 'district_num', 'dem_candidate', 'two_party_vtshare',
       'county_num', 'office', 'urban', 'border'],
      dtype='object')

In [11]:
for_house.border.value_counts()

0    171
Name: border, dtype: int64

In [12]:
for_house.two_party_vtshare.describe()

count    171.000000
mean       0.394659
std        0.224441
min        0.056778
25%        0.208275
50%        0.337235
75%        0.556374
max        0.956403
Name: two_party_vtshare, dtype: float64

In [13]:
demc = for_house[for_house.two_party_vtshare > .5]

In [14]:
demc.border.value_counts()

0    49
Name: border, dtype: int64

In [15]:
for_house[for_house.border == 1]

,county,district_num,dem_candidate,two_party_vtshare,county_num,office,urban,border


In [16]:
dem_districcts = for_house.groupby('district_num')['two_party_vtshare'].median()

In [18]:
for_house.sort_values(by='county')

,county,district_num,dem_candidate,two_party_vtshare,county_num,office,urban,border
0,ANGELINA,1,Shirley McKellar,0.324669,3,house,0,0
134,ARANSAS,27,Raul (Roy) Barrera,0.243980,4,house,1,0
149,ATASCOSA,28,Henry Cuellar,0.470437,7,house,1,0
31,AUSTIN,10,Tawana W. Cadien,0.162999,8,house,1,0
73,BANDERA,21,Tom Wakely,0.158058,10,house,1,0
34,BASTROP,27,Raul (Roy) Barrera,0.383187,11,house,1,0
33,BASTROP,17,William Matta,0.306494,11,house,1,0
32,BASTROP,10,Tawana W. Cadien,0.382285,11,house,1,0
155,BEE,34,Filemon B. Vela,0.434325,13,house,0,0
124,BELL,31,Mike Clark,0.352228,14,house,1,0


In [19]:
for_pres

,county,county_num,office,dem_candidate,two_party_vtshare,candidate_male,candidate_white,candidate_hispanic,urban,border
0,ANDERSON,1,President,Hillary Clinton,0.203319,0,1,0,Rural,Non-Border
1,ANDREWS,2,President,Hillary Clinton,0.175520,0,1,0,Rural,Non-Border
2,ANGELINA,3,President,Hillary Clinton,0.258098,0,1,0,Rural,Non-Border
3,ARANSAS,4,President,Hillary Clinton,0.241548,0,1,0,Urban,Non-Border
4,ARCHER,5,President,Hillary Clinton,0.094258,0,1,0,Urban,Non-Border
5,ARMSTRONG,6,President,Hillary Clinton,0.070423,0,1,0,Urban,Non-Border
6,ATASCOSA,7,President,Hillary Clinton,0.350516,0,1,0,Urban,Non-Border
7,AUSTIN,8,President,Hillary Clinton,0.194029,0,1,0,Urban,Non-Border
8,BAILEY,9,President,Hillary Clinton,0.228030,0,1,0,Rural,Non-Border
9,BANDERA,10,President,Hillary Clinton,0.174537,0,1,0,Urban,Non-Border
